In [1]:
import fitz
import json
import base64

import cv2

In [2]:
def get_rect_box(data):
    if data['type'] == 'text':
        width = fitz.getTextlength(text=data['text'], fontsize=int(data['fontSize'][:-2])) + 2
        height = int(data['fontSize'][:-2]) + 2
    elif data['type'] == 'img':
        width = float(data['width'][:-2])
        height = float(data['height'][:-2])
        
    rect_x1 = float(data['left'][:-2])
    rect_y1 = float(data['top'][:-2])
    rect_x2 = rect_x1 + width
    rect_y2 = rect_y1 + height
    
    return rect_x1, rect_y1, rect_x2, rect_y2

In [3]:
def fill_pdf(input_data_file, original_file, filled_result_dir):
    # read input data and original pdf     
    input_data = json.load(open(input_data_file, 'r'))
    if original_file.split('.')[-1].lower() == 'pdf':
        doc = fitz.open(original_file)
    else:
        img = cv2.imread(original_file)
        src = fitz.open('pdf', fitz.open('1-img/1.jpg').convertToPDF())
        doc = fitz.open()
        page = doc.newPage(width = img.shape[1], height = img.shape[0])
        page.showPDFpage(page.rect, src)
    
    # fill data     
    for data in input_data:
        page = doc[int(data['page']) - 1]
        if data['type'] == 'text':
            page.insertTextbox(get_rect_box(data), data['text'], fontsize=int(data['fontSize'][:-2]), color=(0,0,0))
        elif data['type'] == 'img':
            sig_img = filled_result_dir + data['page'] + '-' + data['id'] + '.png'
            open(sig_img, 'wb').write(base64.b64decode(data['img'].replace('data:image/png;base64,', '')))
            page.insertImage(get_rect_box(data), sig_img)
    doc.save(filled_result_dir + 'filled.pdf')

In [4]:
fill_pdf(input_data_file='1-img/input.json', original_file='1-img/1.jpg', filled_result_dir='1-img/')